In [41]:
%pip install -q dagshub mlflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import numpy as np
import mlflow
import dagshub
import mlflow.sklearn

In [17]:
dagshub.init(repo_owner='Xxwdsdfs', repo_name='ml-production-app', mlflow=True)

Initialized MLflow to track repo "Xxwdsdfs/ml-production-app"

Repository Xxwdsdfs/ml-production-app initialized!

In [3]:
# Vérifier l'URI de suivi et l'expérience en cours
print("MLflow Tracking URI:", mlflow.get_tracking_uri())

MLflow Tracking URI: https://dagshub.com/Xxwdsdfs/ml-production-app.mlflow


In [6]:
pip install --user scikit-learn==1.5.1

Note: you may need to restart the kernel to use updated packages.


In [6]:
print("Version de scikit-learn :", sklearn.__version__)

Version de scikit-learn : 1.5.1


In [4]:
# Charger les données
fichier_csv = "../../data/blackjack_data.csv"  # À ajuster selon l'emplacement
data = pd.read_csv(fichier_csv)

In [5]:
# Afficher les premières lignes
print("Aperçu des données :")
print(data.head())

Aperçu des données :
   Total_Joueur  Carte_Croupier  Action
0            14              10   tirer
1             9               9   tirer
2            13               3  rester
3            20               2  rester
4            12               7   tirer


In [6]:
# Vérifier les informations générales
print("\nInformations sur les données :")
print(data.info())


Informations sur les données :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Total_Joueur    10000 non-null  int64 
 1   Carte_Croupier  10000 non-null  int64 
 2   Action          10000 non-null  object
dtypes: int64(2), object(1)
memory usage: 234.5+ KB
None


In [7]:
# Vérifier les valeurs manquantes
print("\nValeurs manquantes :")
print(data.isnull().sum())


Valeurs manquantes :
Total_Joueur      0
Carte_Croupier    0
Action            0
dtype: int64


In [8]:
# Vérifier la distribution des actions
print("\nDistribution des actions :")
print(data['Action'].value_counts())


Distribution des actions :
tirer     5178
rester    4822
Name: Action, dtype: int64


In [9]:
# Encodage de la colonne "Action"
encoder = LabelEncoder()
data['Action'] = encoder.fit_transform(data['Action'])  # "tirer" -> 1, "rester" -> 0

In [10]:
# Définir les features et le label
X = data[['Total_Joueur', 'Carte_Croupier']]  # Entrées
y = data['Action']  # Sortie

In [11]:
# Diviser en ensemble d'entraînement et de test (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
print("\nDonnées préparées :")
print(f"Taille de l'ensemble d'entraînement : {X_train.shape[0]} exemples")
print(f"Taille de l'ensemble de test : {X_test.shape[0]} exemples")


Données préparées :
Taille de l'ensemble d'entraînement : 8000 exemples
Taille de l'ensemble de test : 2000 exemples


In [15]:
# Exemple d'entrée
example_input = pd.DataFrame([[14, 10]], columns=['Total_Joueur', 'Carte_Croupier'])

In [16]:
with mlflow.start_run():
    
    # Initialiser et entraîner le modèle
    model = DecisionTreeClassifier(max_depth=5)
    model.fit(X_train, y_train)

    # Prédire sur l'ensemble de test
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Loguer les métriques
    mlflow.log_metric("accuracy", accuracy)

    # Loguer les paramètres du modèle
    mlflow.log_param("max_depth", model.max_depth)

# Créer une signature du modèle
    input_schema = mlflow.models.signature.Schema([
        mlflow.models.signature.ColSpec("double", "Total_Joueur"),
        mlflow.models.signature.ColSpec("double", "Carte_Croupier")
    ])
    
    output_schema = mlflow.models.signature.Schema([
        mlflow.models.signature.ColSpec("string")
    ])

    signature = mlflow.models.signature.ModelSignature(inputs=input_schema, outputs=output_schema)

    # Loguer le modèle avec un exemple d'entrée et la signature
    mlflow.sklearn.log_model(model, "model_final", signature=signature, input_example=example_input)

    print("Modèle sauvegardé avec succès !")

Modèle sauvegardé avec succès !
🏃 View run shivering-tern-637 at: https://dagshub.com/Xxwdsdfs/ml-production-app.mlflow/#/experiments/0/runs/ea7fd4ec1d6448fd9dc4db33d7db537a
🧪 View experiment at: https://dagshub.com/Xxwdsdfs/ml-production-app.mlflow/#/experiments/0


In [18]:
# Évaluer les performances
print("\nÉvaluation du modèle :")
print("Précision :", accuracy_score(y_test, y_pred))
print("\nRapport de classification :")
print(classification_report(y_test, y_pred))


Évaluation du modèle :
Précision : 0.9665

Rapport de classification :
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       970
           1       1.00      0.93      0.97      1030

    accuracy                           0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000



In [19]:
# Sauvegarder le modèle dans le dossier 'ml/'
joblib.dump((model, encoder), "../model_registry/model.joblib")
print("Modèle sauvegardé avec succès !")

Modèle sauvegardé avec succès !


In [20]:
model, encoder = joblib.load("../model_registry/model.joblib")

# Exemple de prédiction
exemple = pd.DataFrame([[15, 6]], columns=['Total_Joueur', 'Carte_Croupier'])
prediction = model.predict(exemple)

# Décoder la prédiction pour obtenir "tirer" ou "rester"
action = encoder.inverse_transform(prediction)
print("Prédiction :", action[0])

Prédiction : rester


In [21]:
import sklearn
print(sklearn.__version__)

1.5.1
